In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers sentencepiece nltk accelerate protobuf
!pip install ipywidgets notebook jupyterlab


In [ ]:
import nltk
import heapq
import torch
import gc
from transformers import pipeline

def extractive_summarization(text, num_sentences=3):
    nltk.download('punkt')
    nltk.download('stopwords')
    
    sentences = nltk.sent_tokenize(text)
    word_frequencies = {}
    stopwords = set(nltk.corpus.stopwords.words("english"))
    
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            if word not in word_frequencies:
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    
    max_freq = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] /= max_freq
    
    sentence_scores = {}
    for sent in sentences:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies:
                if sent not in sentence_scores:
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
    
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    return ' '.join(summary_sentences)

def abstractive_summarization(text, model_name="sshleifer/distilbart-cnn-12-6"):
    gc.collect()
    torch.cuda.empty_cache()
    
    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, device=0 if torch.cuda.is_available() else -1, use_fast=True)
    summary = summarizer(text, max_length=65, min_length=20, do_sample=False)
    
    del summarizer  # Free memory
    gc.collect()
    
    return summary[0]['summary_text']

# Verify GPU availability
print("Is CUDA available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))

# Example usage in Jupyter Notebook:
text = """Autonomous vehicles are transforming transportation by reducing human intervention in driving. These vehicles use sensors, cameras, LiDAR, and AI algorithms to navigate. The goal is to enhance road safety and reduce traffic congestion. However, challenges such as unpredictable human behavior, adverse weather conditions, and ethical considerations in decision-making remain. Despite these hurdles, continuous advancements in AI and machine learning are making self-driving technology more reliable and accessible."""

print("Extractive Summary:")
print(extractive_summarization(text))

print("\nAbstractive Summary:")
print(abstractive_summarization(text))
